## The goal of this notebook is to produce an exploitable csv for visualizing the longitudinality of the global questions

# Longitudinal Analysis of Global Survey Questions

This notebook creates a ready-to-use CSV that links each survey question back to its overarching “global” theme and records when it first appeared—perfect for plotting how key questions have evolved over time. We:

1. **Load inputs**  
   - Combined raw responses per commune (`commune_responses_combined_raw.csv`)  
   - Metadata on our top 10 “global” questions (`top_10_QuestionGlobales_NLP.csv`)  
   - Full list of global question labels (`QuestionGlobales_NLP.csv`)

2. **Annotate question metadata**  
   - Extract the first survey year for each global question code  
   - Explode multi-mapped codes to build a mapping dictionary from any sub-question back to its global parent

3. **Link responses to global questions**  
   - Transpose the response table so question codes become rows  
   - Derive a `year` column from each question’s code (e.g., GSB21 → 2021)  
   - Map every question in the dataset to its `quest_glob` parent code

4. **Standardize multilingual labels**  
   - Identify all “spr” columns (label translations)  
   - Replace each label with the respondent’s chosen survey language for consistency

5. **Export for visualization**  
   - Save the final table as `commune_responses_combined.csv`, ready for longitudinal plotting of global question trends


In [27]:
import pandas as pd 
import numpy as np
import re

In [28]:
df_commune_responses_combined = pd.read_csv("../data/commune_responses_combined_raw.csv", index_col=False)
top_10_questions_globales = pd.read_csv("../data/top_10_QuestionGlobales_NLP.csv")
full_question_globale_NLP = pd.read_csv("../data/QuestionGlobales_NLP.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_13132\3964980720.py:1: DtypeWarning: Columns (0,3,8,10,32,61,62,65,68,70,72,74,76,80,81,82,89,90,91,92,93,94,98,99,100,101,102,104,105,107,108,179,180,181,182,183,184,185,186,187,188,189,190,637,638,639,640,641,642) have mixed types. Specify dtype option on import or set low_memory=False.
  df_commune_responses_combined = pd.read_csv("../data/commune_responses_combined_raw.csv", index_col=False)


In [29]:
start_col = df_commune_responses_combined.columns.get_loc("GSB23_Q99")
df_commune_responses_combined = df_commune_responses_combined.iloc[:, start_col:]
df_commune_responses_combined.head(1)

,GSB23_Q99,GSB23_Q100,GSB23_Q101,GSB23_Q10,GSB23_Q4,GSB23_Q5,GSB23_Q6,GSB23_Q7,GSB23_Q8,GSB23_Q9,...,GSB09_30_1,GSB09_30_2,GSB09_30_3,GSB09_31A,GSB09_31B,GSB09_32,GSB09_33,GSB09_34A_1,GSB09_34A_2,GSB09_34A_3
0,1.0,1.0,NaN,1.0,1.0,3.0,2.0,1.0,-2.0,3.0,...,,,,,,,,,,


In [30]:
# on ajoute une colonne pour chaque question globale avec l'année de la question code_first_question
top_10_questions_globales["first_year"] = top_10_questions_globales["code_first_question"].str.extract(r'GSB(\d{2})').astype(float) + 1900
top_10_questions_globales["first_year"] = top_10_questions_globales["first_year"].apply(lambda x: x if x >= 1950 else x + 100).astype(int)

In [31]:
top_10_questions_globales.head(1)

,label,code_first_question,code_other_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label,first_year
0,arbeiten,GSB23_Q52,GSB17_Q58,Arbeitet der/die Gemeindepräsident/-in aktiv i...,Le maire participe-t-il activement à l'adminis...,Il sindaco partecipa attivamente all'amministr...,Not available for the moment,Does the mayor actively participate in the adm...,NaN,NaN,NaN,NaN,NaN,NaN,2.0;1.0;-99.0,NaN,2023


transpose le df_combined et rajouter 2 colonnes : 1 qui contient l'année du survey et une colonne qui contient l'id unique de la question globale (si y'en a une associée)

In [32]:
df_commune_responses_combined = df_commune_responses_combined.transpose()

In [33]:
df_commune_responses_combined = df_commune_responses_combined.iloc[:, :3197]
df_commune_responses_combined.head()

,0,1,2,3,4,5,6,7,8,9,...,3187,3188,3189,3190,3191,3192,3193,3194,3195,3196
GSB23_Q99,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSB23_Q100,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSB23_Q101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSB23_Q10,1.0,2.0,2.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSB23_Q4,1.0,2.0,2.0,2.0,3.0,2.0,1.0,1.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
def extract_first_two_digits(id_str):
    if (id_str.startswith("GSB")) or (id_str.startswith("spr")):
        match = re.search(r'\d{2}', id_str)  # Cherche les deux premiers chiffres
        return match.group(0) if match else None
    else:
        # Si l'ID ne commence pas par "GSB", on cherche 4 chiffres collés
        match = re.search(r'(\d{4})', id_str)  # Cherche un groupe de 4 chiffres
        return match.group(0) if match else None

In [35]:
df_commune_responses_combined["year"] = df_commune_responses_combined.index.map(extract_first_two_digits)
df_commune_responses_combined["year"] = df_commune_responses_combined["year"].apply(
    lambda x: int(x) if pd.notna(x) and len(str(x)) == 4 else (
        (int(x) + 1900) if pd.notna(x) and len(str(x)) == 2 and int(x) >= 50 else (
            (int(x) + 2000) if pd.notna(x) and len(str(x)) == 2 else None
        )
    )
)

df_commune_responses_combined['year'] = df_commune_responses_combined['year'].fillna(-1).astype(int)
df_commune_responses_combined['year'] = df_commune_responses_combined['year'].replace(-1, pd.NA)



df_commune_responses_combined.head()




,0,1,2,3,4,5,6,7,8,9,...,3188,3189,3190,3191,3192,3193,3194,3195,3196,year
GSB23_Q99,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023
GSB23_Q100,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023
GSB23_Q101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023
GSB23_Q10,1.0,2.0,2.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023
GSB23_Q4,1.0,2.0,2.0,2.0,3.0,2.0,1.0,1.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023


## add column of global question associate 

In [36]:
top_10_questions_globales.head(1)

,label,code_first_question,code_other_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label,first_year
0,arbeiten,GSB23_Q52,GSB17_Q58,Arbeitet der/die Gemeindepräsident/-in aktiv i...,Le maire participe-t-il activement à l'adminis...,Il sindaco partecipa attivamente all'amministr...,Not available for the moment,Does the mayor actively participate in the adm...,NaN,NaN,NaN,NaN,NaN,NaN,2.0;1.0;-99.0,NaN,2023


In [37]:
# Exploser les valeurs multiples dans `code_other_question`
exploded_top_10 = top_10_questions_globales.assign(
    code_other_question=top_10_questions_globales['code_other_question'].str.split('; ')
).explode('code_other_question')

# Créer le dictionnaire de correspondance pour `code_other_question`
other_question_mapping = exploded_top_10.set_index('code_other_question')['code_first_question'].to_dict()

# Créer self_mapping en accédant directement à la colonne sans set_index
self_mapping = dict(zip(top_10_questions_globales['code_first_question'], top_10_questions_globales['code_first_question']))

# Combiner les deux dictionnaires
mapping_dict = {**other_question_mapping, **self_mapping}

# Afficher le dictionnaire final pour vérification
print(mapping_dict)


{'GSB17_Q58': 'GSB23_Q52', 'GSB05_38': 'GSB94_27', 'GSB09_34': 'GSB94_27', 'GSB05_33': 'GSB94_25', 'GSB05_37': 'GSB94_38a', 'GSB94_21': 'GSB88_52', 'GSB05_29': 'GSB88_52', 'GSB05_50': 'GSB94_36', 'GSB05_35': 'GSB94_24', 'GSB17_Q7': 'GSB23_Q9', 'GSB17_Q6': 'GSB23_Q7', 'spr88': 'GSB23_UserLanguage', 'spr94': 'GSB23_UserLanguage', 'spr98': 'GSB23_UserLanguage', 'spr05': 'GSB23_UserLanguage', 'spr09': 'GSB23_UserLanguage', 'spr17': 'GSB23_UserLanguage', 'GSB23_Q52': 'GSB23_Q52', 'GSB94_27': 'GSB94_27', 'GSB94_25': 'GSB94_25', 'GSB94_38a': 'GSB94_38a', 'GSB88_52': 'GSB88_52', 'GSB94_36': 'GSB94_36', 'GSB94_24': 'GSB94_24', 'GSB23_Q9': 'GSB23_Q9', 'GSB23_Q7': 'GSB23_Q7', 'GSB23_UserLanguage': 'GSB23_UserLanguage'}


In [38]:
df_commune_responses_combined['quest_glob'] = df_commune_responses_combined.index.map(mapping_dict)


In [39]:
df_commune_responses_combined.head()

,0,1,2,3,4,5,6,7,8,9,...,3189,3190,3191,3192,3193,3194,3195,3196,year,quest_glob
GSB23_Q99,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,NaN
GSB23_Q100,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,NaN
GSB23_Q101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,NaN
GSB23_Q10,1.0,2.0,2.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,NaN
GSB23_Q4,1.0,2.0,2.0,2.0,3.0,2.0,1.0,1.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,NaN


In [40]:
df_commune_responses_combined = df_commune_responses_combined.T

In [41]:
# Sélectionner toutes les colonnes commençant par "spr"
spr_columns = [col for col in df_commune_responses_combined.columns if col.startswith("spr")]

# Remplacer les valeurs des colonnes "spr" par les valeurs de "GSB23_UserLanguage", sauf pour la ligne "year"
for col in spr_columns:
    df_commune_responses_combined.loc[df_commune_responses_combined.index != 'year', col] = df_commune_responses_combined["GSB23_UserLanguage"]

In [42]:
spr_columns = [col for col in df_commune_responses_combined.columns if 'spr' in col]
print(df_commune_responses_combined[spr_columns])

                         spr88               spr94               spr98  \
0                          1.0                 1.0                 1.0   
1                          1.0                 1.0                 1.0   
2                          1.0                 1.0                 1.0   
3                          1.0                 1.0                 1.0   
4                          1.0                 1.0                 1.0   
...                        ...                 ...                 ...   
3194                       NaN                 NaN                 NaN   
3195                       NaN                 NaN                 NaN   
3196                       NaN                 NaN                 NaN   
year                      1988                1994                1998   
quest_glob  GSB23_UserLanguage  GSB23_UserLanguage  GSB23_UserLanguage   

                         spr05               spr09               spr17  
0                          1.0        

In [44]:
df_commune_responses_combined.to_csv('../data/commune_responses_combined.csv')